In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

# Merge datasets
book_ratings = pd.merge(ratings, books, on='book_id')
book_user_ratings = pd.merge(book_ratings, users, on='user_id')

# Create user-item matrix
user_item_matrix = pd.pivot_table(book_user_ratings, values='rating', index='user_id', columns='book_id')

# Replace NaN values with 0 (indicating no rating)
user_item_matrix = user_item_matrix.fillna(0)

# Calculate item-item similarity matrix using cosine similarity
item_similarity = cosine_similarity(user_item_matrix.T)

# Function to get top N recommendations for a user
def get_top_n_recommendations(user_id, n=10):
    user_ratings = user_item_matrix.loc[user_id]
    similar_scores = item_similarity.dot(user_ratings)
    similar_scores = similar_scores / np.array([np.abs(item_similarity).sum(axis=1)]).T  # Normalize
    sorted_indices = np.argsort(similar_scores)[::-1]
    
    # Exclude books already rated by the user
    user_rated_books = set(book_user_ratings[book_user_ratings['user_id'] == user_id]['book_id'])
    recommendations = []
    for index in sorted_indices:
        if index not in user_rated_books:
            recommendations.append(index)
        if len(recommendations) >= n:
            break
            
    top_n_books = books[books['book_id'].isin(recommendations)]['title']
    
    return top_n_books

# Example usage
user_id = 1
top_n_recommendations = get_top_n_recommendations(user_id)
print(f"Top 10 recommendations for user {user_id}:")
print(top_n_recommendations)


C:\Users\Afshan Perween\AppData\Local\Temp\ipykernel_24420\606077233.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('books.csv')


KeyError: 'book_id'